In [1]:
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [2]:
from class_trial import StockData

In [3]:
# List of tickers
tic = ["069500.KS", "229200.KS",
             "133690.KS", "143850.KS",
             "114260.KS", "153130.KS",
             "157450.KS"
             ]

# Initializae
stock_data = StockData()

In [4]:
# Get data
data = stock_data.download_data(tic, '2000-01-01', '2022-12-31','1d')

[*********************100%%**********************]  7 of 7 completed


In [5]:
df = stock_data.get_monthly(data)

In [6]:
df

Ticker,069500.KS,114260.KS,133690.KS,143850.KS,153130.KS,157450.KS,229200.KS
Date,,,,,,,
2015-10-30,25075.0,55840.0,23860.0,24265.0,101570.0,101550.0,10645.0
2015-11-30,24490.0,55665.0,24300.0,24185.0,101625.0,101640.0,10980.0
2015-12-30,24345.0,55660.0,24695.0,24105.0,100030.0,100050.0,11015.0
2016-01-29,23525.0,55850.0,22685.0,22130.0,100160.0,100160.0,11280.0
2016-02-29,23930.0,56145.0,23395.0,22635.0,100305.0,100285.0,10440.0
...,...,...,...,...,...,...,...
2022-07-29,32385.0,56855.0,74890.0,48080.0,103505.0,101040.0,11390.0
2022-08-31,32230.0,55870.0,74495.0,46985.0,103590.0,101175.0,11395.0
2022-09-30,28275.0,55280.0,71780.0,42735.0,103735.0,101380.0,9535.0


In [7]:
from technical_indicators import technicalindicator

In [8]:
nasdaq = stock_data.download_data(['^IXIC'],'2000-01-01', '2022-12-31','1d')
df_nasdaq = stock_data.get_monthly(nasdaq)

[*********************100%%**********************]  1 of 1 completed


In [9]:
nasdaq_macd = technicalindicator.calculate_macd(df_nasdaq)[0]
nasdaq_5ma = technicalindicator.get_moving_average(df_nasdaq,5)
nasdaq_20ma = technicalindicator.get_moving_average(df_nasdaq,20)
merged_df = pd.merge(df_nasdaq, nasdaq_5ma, left_index=True, right_index=True, how='outer')
merged_df1 = pd.merge(merged_df, nasdaq_20ma, left_index=True, right_index=True, how='outer')
merged_df1 = pd.merge(merged_df1, nasdaq_macd, left_index=True, right_index=True, how='outer')

merged_df1.rename(columns={'Close_x': 'Close', 'Close_ma5_x': '5mo MA', 
                            'Close_ma5_y': '20mo MA', 'Close_y':'MACD'}, inplace=True)

nasdaq_ind = merged_df1.dropna()
nasdaq_ind['gc'] = nasdaq_ind['5mo MA'] > nasdaq_ind['20mo MA']
nasdaq_ind['gc_ind'] = (nasdaq_ind.gc - nasdaq_ind.gc.shift(1)).replace(-1, 0)
nasdaq_ind['gc_sum'] = nasdaq_ind['gc_ind'].rolling(window=6).sum()

C:\Users\ewp\AppData\Local\Temp\ipykernel_71776\2009895064.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nasdaq_ind['gc'] = nasdaq_ind['5mo MA'] > nasdaq_ind['20mo MA']
C:\Users\ewp\AppData\Local\Temp\ipykernel_71776\2009895064.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  nasdaq_ind['gc_ind'] = (nasdaq_ind.gc - nasdaq_ind.gc.shift(1)).replace(-1, 0)
C:\Users\ewp\AppData\Local\Temp\ipykernel_71776\2009895064.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [10]:
nasdaq_ind

,Close,5mo MA,20mo MA,MACD,gc,gc_ind,gc_sum
Date,,,,,,,
2001-08-31,1805.430054,2043.966016,3075.322009,-536.955161,False,NaN,NaN
2001-09-28,1498.800049,1920.478027,2953.244507,-571.761672,False,0.0,NaN
2001-10-31,1690.199951,1836.420020,2802.920007,-577.247560,False,0.0,NaN
2001-11-30,1930.579956,1790.428003,2670.807501,-555.791688,False,0.0,NaN
2001-12-31,1950.400024,1775.082007,2575.294507,-531.066635,False,0.0,NaN
...,...,...,...,...,...,...,...
2022-07-29,12390.690430,12411.195898,13786.118506,474.491977,False,0.0,0.0
2022-08-31,11816.200195,11930.332031,13732.514502,336.444240,False,0.0,0.0
2022-09-30,10575.620117,11578.528125,13607.760986,125.489406,False,0.0,0.0


In [11]:
def get_moving_average(df, period):
    ma = df.rolling(period).mean().dropna()
    ma.columns = [f'{col}_ma5' for col in ma.columns]
    return ma

In [12]:
ma = get_moving_average(df,5)

In [13]:
ma

,069500.KS_ma5,114260.KS_ma5,133690.KS_ma5,143850.KS_ma5,153130.KS_ma5,157450.KS_ma5,229200.KS_ma5
Date,,,,,,,
2016-02-29,24273.0,55832.0,23787.0,23464.0,100738.0,100737.0,10872.0
2016-03-31,24249.0,55872.0,23626.0,23415.0,100508.0,100509.0,10893.0
2016-04-29,24241.0,55955.0,23243.0,23416.0,100290.0,100285.0,10821.0
2016-05-31,24232.0,56042.0,23166.0,23521.0,100409.0,100402.0,10743.0
2016-06-30,24407.0,56121.0,23156.0,23946.0,100540.0,100528.0,10551.0
...,...,...,...,...,...,...,...
2022-07-29,34258.0,56432.0,73941.0,49124.0,103407.0,100880.0,11987.0
2022-08-31,33288.0,56248.0,72406.0,47697.0,103464.0,100968.0,11640.0
2022-09-30,31841.0,56038.0,71716.0,46201.0,103537.0,101083.0,11058.0


In [90]:
get_moving_average(df,20)

,069500.KS_ma5,114260.KS_ma5,133690.KS_ma5,143850.KS_ma5,153130.KS_ma5,157450.KS_ma5,229200.KS_ma5
Date,,,,,,,
2017-05-31,25778.75,55881.00,25005.50,25388.75,100671.00,100658.25,10089.50
2017-06-30,26091.25,55864.50,25273.50,25612.50,100629.00,100614.00,10072.00
2017-07-31,26439.75,55859.75,25553.00,25869.75,100590.50,100570.25,10028.75
2017-08-31,26763.00,55857.25,25834.50,26124.75,100636.50,100611.25,9995.00
2017-09-29,27171.75,55830.75,26239.75,26508.00,100680.50,100652.50,9971.25
...,...,...,...,...,...,...,...
2022-07-29,38920.00,57463.75,74168.00,49586.00,103017.75,100433.75,13656.25
2022-08-31,38556.50,57354.50,74752.00,49759.75,103067.00,100492.00,13453.00
2022-09-30,37914.00,57212.50,75083.00,49719.25,103120.00,100558.00,13209.00


In [14]:
def calculate_macd(price_series):
    ema_12 = price_series.ewm(span=12, adjust=False).mean()
    ema_26 = price_series.ewm(span=26, adjust=False).mean()
    macd = ema_12 - ema_26
    signal_line = macd.ewm(span=9, adjust=False).mean()
    return macd, signal_line

In [115]:
macd = []
for stock in df.columns:
    macd.append(calculate_macd(df[stock])[0])
macd_df = pd.DataFrame(macd).T.iloc[1:,:]
macd_df.columns = [f'{col}_ma5' for col in macd_df.columns]

In [116]:
macd_df

,069500.KS_ma5,114260.KS_ma5,133690.KS_ma5,143850.KS_ma5,153130.KS_ma5,157450.KS_ma5,229200.KS_ma5
Date,,,,,,,
2015-11-30,-46.666667,-13.960114,35.099715,-6.381766,4.387464,7.179487,26.723647
2015-12-30,-94.264008,-25.137296,93.709467,-17.690765,-119.461530,-114.114983,50.148457
2016-01-29,-195.894243,-18.451187,-21.781007,-183.899346,-204.762463,-199.070878,89.069337
2016-02-29,-240.978966,10.530250,-55.378603,-271.739124,-257.693251,-253.391641,51.539321
2016-03-31,-191.789174,24.740434,-108.192918,-226.978391,-287.052778,-283.091479,46.277465
...,...,...,...,...,...,...,...
2022-07-29,131.151218,-240.029061,5205.139815,2465.666541,370.058900,108.013782,-194.705531
2022-08-31,-165.351443,-308.845671,4825.494534,2096.495357,372.738588,138.015864,-279.903651
2022-09-30,-711.268327,-406.307737,4256.479063,1444.335650,382.157277,176.302215,-491.840598


---

In [11]:
uni_close = data.Close.dropna(thresh = 7)

In [12]:
test_init = uni_close[ (uni_close.index >= ma.index[0]) & (uni_close.index>= macd_df.index[0])]

NameError: name 'ma' is not defined

In [13]:
test_init

NameError: name 'test_init' is not defined

In [20]:
test1 = (test_init.pct_change()+1)
test1.replace(np.nan, 1, inplace = True)
test1 = test1.reset_index()
test1['Date'] = pd.to_datetime(test1['Date'])
test1.insert(1, 'month', test1.Date.dt.month)
test1.insert(1, 'year', test1.Date.dt.year)
test1.insert(1, 'end of month', (test1.Date.dt.month - test1.Date.shift(-1).dt.month).isin([0,np.nan]))
test1.set_index('Date',inplace= True)
test1

Ticker,end of month,year,month,069500.KS,114260.KS,133690.KS,143850.KS,153130.KS,157450.KS,229200.KS
Date,,,,,,,,,,
2016-02-29,False,2016,2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2016-03-02,True,2016,3,1.015253,0.999555,1.026715,1.024961,1.000100,1.000100,1.018678
2016-03-03,True,2016,3,1.003087,1.003207,0.989800,0.998922,1.000000,1.000100,1.020216
2016-03-04,True,2016,3,1.000205,1.000000,0.985279,1.004315,1.000100,1.000050,1.002765
2016-03-07,True,2016,3,1.002051,0.998757,0.996371,1.000644,1.000000,1.000000,0.988511
...,...,...,...,...,...,...,...,...,...,...
2022-12-23,True,2022,12,0.982773,0.999117,0.976157,0.982953,1.000287,1.000196,0.960236
2022-12-26,True,2022,12,1.001488,0.999470,0.998409,1.006306,1.000000,1.000000,1.018916
2022-12-27,True,2022,12,1.007100,0.998586,1.002869,1.005260,1.000000,1.000098,1.014551


In [21]:
ret = test1.iloc[:,3:]

In [22]:
ret

Ticker,069500.KS,114260.KS,133690.KS,143850.KS,153130.KS,157450.KS,229200.KS
Date,,,,,,,
2016-02-29,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2016-03-02,1.015253,0.999555,1.026715,1.024961,1.000100,1.000100,1.018678
2016-03-03,1.003087,1.003207,0.989800,0.998922,1.000000,1.000100,1.020216
2016-03-04,1.000205,1.000000,0.985279,1.004315,1.000100,1.000050,1.002765
2016-03-07,1.002051,0.998757,0.996371,1.000644,1.000000,1.000000,0.988511
...,...,...,...,...,...,...,...
2022-12-23,0.982773,0.999117,0.976157,0.982953,1.000287,1.000196,0.960236
2022-12-26,1.001488,0.999470,0.998409,1.006306,1.000000,1.000000,1.018916
2022-12-27,1.007100,0.998586,1.002869,1.005260,1.000000,1.000098,1.014551


In [96]:
odd_stock_w = 0.7
odd_bond_w = 0.3
even_stock_w = 0.5
even_bond_w = 0.5
even_passive_stock_w = 0.3
even_passive_bond_w = 0.7
odd_buy = np.array([0.2*odd_stock_w, (1/3*odd_bond_w), 0.3*odd_stock_w, 0.2*odd_stock_w, (1/3*odd_bond_w), (1/3*odd_bond_w), 0.3*odd_stock_w])          # Even 11 - Odd 4
odd_hold = np.array([0.2*odd_stock_w, (1/3*odd_bond_w), 0.3*odd_stock_w, 0.2*odd_stock_w, (1/3*odd_bond_w), (1/3*odd_bond_w), 0.0*odd_stock_w])         # Odd 5 - Odd 10
even_buy = np.array([0.2*even_stock_w, (1/3*even_bond_w), 0.4*even_stock_w, 0.4*even_stock_w, (1/3*even_bond_w), (1/3*even_bond_w), 0.0*even_stock_w])  # Odd 11 - Even 4
even_hold = np.array([0.0*even_passive_stock_w, (1/3*even_passive_bond_w), 0.5*even_passive_stock_w, 0.5*even_passive_stock_w, (1/3*even_passive_bond_w), (1/3*even_passive_bond_w), 0.0*even_passive_stock_w])  # Even 5 - Even 10

In [97]:
logic_sell = np.array([0, (1/3), 0, 0, (1/3), (1/3), 0]) 

In [19]:
nasdaq_ind

,Close,5mo MA,20mo MA,MACD,gc,gc_ind,gc_sum
Date,,,,,,,
2001-08-31,1805.430054,2043.966016,3075.322009,-536.955161,False,NaN,NaN
2001-09-28,1498.800049,1920.478027,2953.244507,-571.761672,False,0.0,NaN
2001-10-31,1690.199951,1836.420020,2802.920007,-577.247560,False,0.0,NaN
2001-11-30,1930.579956,1790.428003,2670.807501,-555.791688,False,0.0,NaN
2001-12-31,1950.400024,1775.082007,2575.294507,-531.066635,False,0.0,NaN
...,...,...,...,...,...,...,...
2022-07-29,12390.690430,12411.195898,13786.118506,474.491977,False,0.0,0.0
2022-08-31,11816.200195,11930.332031,13732.514502,336.444240,False,0.0,0.0
2022-09-30,10575.620117,11578.528125,13607.760986,125.489406,False,0.0,0.0


In [22]:
tt = []
for i in range(len(nasdaq_ind)):
    if test1['end of month'][i] == False: # End of month 
        ti = nasdaq_ind[(nasdaq_ind.index.month == ret.index[i].month)&(nasdaq_ind.index.year == ret.index[i].year)]
    else: # During month 
        index_date = ret.index[i] - relativedelta(months=1)
        ti = nasdaq_ind[(nasdaq_ind.index.month == index_date.month)&(nasdaq_ind.index.year == index_date.year)]
    tt.append(ti)

NameError: name 'test1' is not defined

In [20]:
def algo2(test1,ret,ma,macd_df,nasdaq_ind):
    #########################################################################
    # Halloween Adjusted + Logic: 
        # Sell signal: (Sell all stocks, go all bonds)
            # monthly closing price closes below 5 month moving average 
            # (Exception 1):
                # When 5-month MA > 20-month MA within the past 6 months 
            # (Exception 2):
                # If odd year
        # Re-entry: 
            # monthly closing price closes above 5 month moving average 
    #########################################################################

    port_list = [10000000000]  # Seed money 
    port = port_list[0]

    mkt_val = []
    cash_rem = []
    strat = []

    # Indicator for whether we have entered the market or not
    invested = False
    logic_sell = False 

    # Start Backtest! 
    for i in range(len(test1)):

        if invested == False: 
            # We are not yet invested in any assets. All Cash 
            pass
        else: 
            # Now, we assume that we have # of shares, mkt value of each asset (i.e. we are invested)
            #print("Print daily return", test1['Date'][i])
            new_mkt_value = mkt_value*(ret.iloc[i,:]) # Record returns over the previous period 
            # NOTE: For daily returns, each asset return should be equal to the returns matrix. But, portfolio value may be unique. 
            port = np.sum(new_mkt_value) + cash_remaining # Record new portfolio value 
            port_list.append(new_mkt_value)

            # Daily (Record daily changes in mkt value, shares, and remaining cash)
            # Remaining cash, shares - constant. Mkt value - changing. 
            # Portfolio mkt value - changing (recorded above). 
            if bool(test1['end of month'][i]) == True:
                #shares.append(s.values)
                cash_rem.append(cash_remaining)
                strat.append(x)
                mkt_val.append(new_mkt_value.values)
                mkt_value = new_mkt_value

        # When to rebalance (Rebalance at last trading day of month)

        # First entry.        
        if i == 0: 
            if bool(test1['end of month'][i]) == False:  # If we start at end of the month
                if ((test1.month.values[i] in [10,11,12]) & (test1.year.values[i]%2 == 0)) or ((test1.month.values[i] in [1,2,3]) & (test1.year.values[i]%2 == 1)): # Halloween strategy
                    X = odd_buy 
                    x = "odd aggressive"
                elif (test1.month.values[i] in [4,5,6,7,8,9]) & (test1.year.values[i]%2 == 1): 
                    X = odd_hold 
                    x = "odd passive"
                elif ((test1.month.values[i] in [10,11,12]) & (test1.year.values[i]%2 == 1)) or ((test1.month.values[i] in [1,2,3]) & (test1.year.values[i]%2 == 0)): 
                    X = even_buy 
                    x = "even aggressive"
                elif (test1.month.values[i] in [4,5,6,7,8,9]) & (test1.year.values[i]%2 == 0): 
                    X = even_hold
                    x = "even passive"
            else:                                # If we start in the middle of the month
                if ((test1.month.values[i] in [11,12]) & (test1.year.values[i]%2 == 0)) or ((test1.month.values[i] in [1,2,3,4]) & (test1.year.values[i]%2 == 1)): 
                    X = odd_buy 
                    x = "odd aggressive"
                elif (test1.month.values[i] in [5,6,7,8,9,10]) & (test1.year.values[i]%2 == 1): 
                    X = odd_hold 
                    x = "odd passive"
                elif ((test1.month.values[i] in [11,12]) & (test1.year.values[i]%2 == 1)) or ((test1.month.values[i] in [1,2,3,4]) & (test1.year.values[i]%2 == 0)): 
                    X = even_buy 
                    x = "even aggressive"
                elif (test1.month.values[i] in [5,6,7,8,9,10]) & (test1.year.values[i]%2 == 0): 
                    X = even_hold
                    x = "even passive"
                    
            strat.append(x)
            # Execute rebalance (# of shares of each asset, mkt value of each asset, cash remaining)
            mkt_value = X*port  # Market value (share X price) of each asset we will be holding 
            mkt_val.append(mkt_value)
            cash_remaining = port - np.sum(mkt_value) # Remaining cash in our portfolio 
            cash_rem.append(cash_remaining)
            
            # Tell indicator that we are invested
            invested = True

        # REbalancing 
        else: 
            if bool(test1['end of month'][i]) == False: # Rebalance when end of month
                #print("Rebalance", test1["Date"][i])
                # Determine which strategy to use
                if ((test1.month.values[i] in [10,11,12]) & (test1.year.values[i]%2 == 0)) or ((test1.month.values[i] in [1,2,3]) & (test1.year.values[i]%2 == 1)): # Halloween strategy
                    X = odd_buy 
                    x = "odd aggressive"
                elif (test1.month.values[i] in [4,5,6,7,8,9]) & (test1.year.values[i]%2 == 1): 
                    X = odd_hold 
                    x = "odd passive"
                elif ((test1.month.values[i] in [10,11,12]) & (test1.year.values[i]%2 == 1)) or ((test1.month.values[i] in [1,2,3]) & (test1.year.values[i]%2 == 0)): 
                    X = even_buy 
                    x = "even aggressive"
                elif (test1.month.values[i] in [4,5,6,7,8,9]) & (test1.year.values[i]%2 == 0): 
                    X = even_hold
                    x = "even passive"

            #########################################################################
            # 1) Navigate corresponding MA, MACD points: 
            if test1['end of month'][i] == False: # End of month 
                ti = nasdaq_ind[(nasdaq_ind.index.month == ret.index[i].month)&(nasdaq_ind.index.year == ret.index[i].year)]
            else: # During month 
                index_date = ret.index[i] - relativedelta(months=1)
                ti = nasdaq_ind[(nasdaq_ind.index.month == index_date.month)&(nasdaq_ind.index.year == index_date.year)]

            # 2) SElL LOGIC 
            if (logic_sell == False) & (ti.Close< ti['5mo MA']):
                # Check for exception: 
                # If golden cross happend in the past 6 months or year is odd, then don't sell. 
                if (ti['gc_ind'] == 1) | ti.index.year%2 == 1: 
                    pass
                else:
                    logic_sell = True
                    X = logic_sell
                    x = "Logic Sell"
            elif (logic_sell == True) & (ti.Close >= ti['5mo MA']):
                print("Buy")
                logic_sell = False
            #########################################################################

                strat.append(x)
                # Execute rebalance (# of shares of each asset, mkt value of each asset, cash remaining)
                #s = np.floor_divide((X*port),test1.iloc[i,4:]) # Determine number of shares for each asset! 
                #shares.append(s.values)
                mkt_value = X*port  # Market value (share X price) of each asset we will be holding 
                mkt_val.append(mkt_value)
                cash_remaining = port - np.sum(mkt_value) # Remaining cash in our portfolio 
                cash_rem.append(cash_remaining)

                
    res_mkt_val = pd.DataFrame(mkt_val)
    res_mkt_val.columns = ret.columns + ' mkt val'
    res_mkt_val.index = ret.index[-len(res_mkt_val):]
    res_mkt_val['cash_rem'] = cash_rem
    res_mkt_val['port_val'] = res_mkt_val.iloc[:, :].sum(axis=1)

    res_pct = res_mkt_val[res_mkt_val.columns[:]].div(res_mkt_val[res_mkt_val.columns[-1]], axis=0)
    res_mkt_val['strategy'] = strat
    return res_mkt_val,res_pct

In [21]:
y1 = algo2( test1, ret ,ma,macd_df,nasdaq_ind)

NameError: name 'test1' is not defined

In [216]:
y1[0]

Ticker,069500.KS mkt val,114260.KS mkt val,133690.KS mkt val,143850.KS mkt val,153130.KS mkt val,157450.KS mkt val,229200.KS mkt val,cash_rem,port_val,strategy
Date,,,,,,,,,,
2016-02-29,1.000000e+09,1.666667e+09,2.000000e+09,2.000000e+09,1.666667e+09,1.666667e+09,0.000000e+00,0.000002,1.000000e+10,even aggressive
2016-03-02,1.015253e+09,1.665925e+09,2.053430e+09,2.049923e+09,1.666833e+09,1.666833e+09,0.000000e+00,0.000002,1.011820e+10,even aggressive
2016-03-03,1.018387e+09,1.671268e+09,2.032486e+09,2.047714e+09,1.666833e+09,1.666999e+09,0.000000e+00,0.000002,1.010369e+10,even aggressive
2016-03-04,1.018596e+09,1.671268e+09,2.002565e+09,2.056550e+09,1.666999e+09,1.667082e+09,0.000000e+00,0.000002,1.008306e+10,even aggressive
2016-03-07,1.020685e+09,1.669190e+09,1.995298e+09,2.057875e+09,1.666999e+09,1.667082e+09,0.000000e+00,0.000002,1.007713e+10,even aggressive
...,...,...,...,...,...,...,...,...,...,...
2022-12-23,2.113650e+09,1.605173e+09,3.110729e+09,2.158188e+09,1.605318e+09,1.605169e+09,3.185883e+09,0.000002,1.538411e+10,odd aggressive
2022-12-26,2.116796e+09,1.604322e+09,3.105780e+09,2.171798e+09,1.605318e+09,1.605169e+09,3.246147e+09,0.000002,1.545533e+10,odd aggressive
2022-12-27,2.131826e+09,1.602053e+09,3.114689e+09,2.183221e+09,1.605318e+09,1.605326e+09,3.293382e+09,0.000002,1.553581e+10,odd aggressive
